In [2]:
import os
os.chdir('../')
from preprocessing import mfcc_extraction as mfcc
from multimodal_net.models import my_alexnet
from multimodal_net import datasets
from multimodal_net import video_transforms
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from torchvision import transforms
import torch
from sklearn.metrics import confusion_matrix


In [4]:
audio_dir = '/mnt/cold/inputs/jersey_number_recognition/letsdance/audio_mfcc'
val_split_file = 'multimodal_net/datasets/letsdance_splits/test.csv'
classes = os.listdir(audio_dir)

In [5]:
fig=plt.figure(figsize=(20, 20))
samples=[]
for ids, cls in enumerate(classes):
    class_files= os.listdir(os.path.join(audio_dir, cls))
    img = mpimg.imread(os.path.join(audio_dir, cls, class_files[np.random.randint(0, len(class_files))]))
    fig.add_subplot(4, 4, ids+1, title=cls)

    plt.imshow(img)




In [11]:
best_model = 'multimodal_net/checkpoints/model_best.pth.tar'
model = my_alexnet.build_model(3, 16)
model.load_state_dict(torch.load(best_model)['state_dict'])
model.eval()


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(

In [43]:
val_dataset = datasets.Letsdance_audio(root='/data/trackml/letsdance/',source=val_split_file,
                                                  phase="val",
                                                  is_color=True,
                                                  new_length=1,
                                                  video_transform=transforms.Compose([video_transforms.Resize((224, 224)),
                                                                                      video_transforms.ToTensor()]))
val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=32, shuffle=True, pin_memory=True)

['ballet', 'break', 'cha', 'flamenco', 'foxtrot', 'jive', 'latin', 'pasodoble', 'quickstep', 'rumba', 'samba', 'square', 'swing', 'tango', 'tap', 'waltz']
